## Population 2016 (IRIS to DEPARTEMENT) - France
infra-communales

In [1]:
import pandas as pd
import numpy as np

### Si besoin de comprendre les variables (nom des colonnes).

In [2]:
# ici sont les variables et leurs explications.
# varpopIR = pd.read_excel('IRIS_population/variable_struct-pop-2016-cIA.xlsx')
# varpopIR

### Récupération des données France (hs Mayotte) et Collect. Outre-Mer

In [3]:
# population France hors Mayotte
franceDEPT = pd.read_excel('IRIS_population/base-ic-evol-struct-pop-2016-cIA.xls')

# population collectivité d'outre-mer (St-Pierre-Miquelon, St-Martin, St-Barthélemy)
popcomIR = pd.read_excel('IRIS_population/base-ic-evol-struct-pop-2016-com-cIA.xls')

In [4]:
print(franceDEPT.shape, popcomIR.shape)

(49637, 84) (16, 80)


### Volume de la population

In [5]:
print("\nTotal Population hors Mayotte et CO-M:", "{:,.2f}".format(franceDEPT['P16_POP'].sum()),
      "\nTotal Hommes hors Mayotte et CO-M :", "{:,.2f}".format(franceDEPT['P16_POPH'].sum()),
      "\nTotal Femmes hors Mayotte et CO-M :", "{:,.2f}".format(franceDEPT['P16_POPF'].sum()),
      "\n\nPopulation par IRIS")

for i in franceDEPT['TYP_IRIS'].unique():
    print("IRIS Type",i, "{:,.2f}".format(franceDEPT.loc[franceDEPT['TYP_IRIS'] == i]['P16_POP'].sum()))


Total Population hors Mayotte et CO-M: 66,361,587.27 
Total Hommes hors Mayotte et CO-M : 32,112,732.25 
Total Femmes hors Mayotte et CO-M : 34,248,855.02 

Population par IRIS
IRIS Type Z 26,955,406.00
IRIS Type H 38,719,713.24
IRIS Type A 443,482.46
IRIS Type D 242,985.57


In [6]:
popmetIR = franceDEPT.copy()

In [7]:
# drop les colonnes de France qui n'apparaissent pas dans le fichier de la Collect. OM.
popmetIR = popmetIR.drop(columns=['REG', 'UU2010', 'TYP_IRIS', 'MODIF_IRIS'], axis=1)
popmetIR.shape

(49637, 80)

#### Merge du fichier France à celui des Collectivités d'Outre-Mer

In [8]:
import warnings
warnings.filterwarnings('ignore')

"""Ajout d'une colonne 'source' qui permet de faire la différence entre fichier France
   et Collectivités d'Outre-Mer."""

popmetIR['source'] = "france"
popcomIR['source'] = "coll_OM"

popglob = [popmetIR, popcomIR]
popglobIR = pd.concat(popglob)
print("popglobIR shape:", popglobIR.shape)

popglobIR shape: (49653, 81)


In [9]:
popglobIR.head()

,IRIS,DEP,COM,LIBCOM,TRIRIS,GRD_QUART,LIBIRIS,LAB_IRIS,P16_POP,P16_POP0002,...,C16_F15P_CS5,C16_F15P_CS6,C16_F15P_CS7,C16_F15P_CS8,P16_POP_FR,P16_POP_ETR,P16_POP_IMM,P16_PMEN,P16_PHORMEN,source
0,010010000,01,01001,L'Abergement-Clémenciat,ZZZZZZ,0100100,L'Abergement-Clémenciat (commune non irisée),5,767.000000,29.000000,...,85.000000,30.000000,70.000000,15.000000,759.000000,8.000000,20.000000,767.000000,0.000000,france
1,010020000,01,01002,L'Abergement-de-Varey,ZZZZZZ,0100200,L'Abergement-de-Varey (commune non irisée),5,243.000000,14.000000,...,25.000000,0.000000,20.000000,0.000000,241.000000,2.000000,3.000000,243.000000,0.000000,france
2,010040101,01,01004,Ambérieu-en-Bugey,ZZZZZZ,0100401,Les Perouses-Triangle d'Activité,1,1893.112206,102.492841,...,158.028018,59.452099,216.381754,195.032628,1614.595359,278.516847,308.098676,1557.106066,336.006141,france
3,010040102,01,01004,Ambérieu-en-Bugey,ZZZZZZ,0100401,Longeray-Gare,1,3546.210164,176.709046,...,354.962018,158.716105,351.399678,346.514489,3115.756603,430.453561,515.488156,3546.210164,0.000000,france
4,010040201,01,01004,Ambérieu-en-Bugey,ZZZZZZ,0100402,Centre-St Germain-Vareilles,1,4100.003588,175.616863,...,407.311067,147.066671,529.371210,334.262710,3831.463866,268.539722,341.362897,4012.001980,88.001608,france


In [10]:
# verification de types de données IRIS après merge
popglobIR['IRIS'].apply(type).unique()

array([<class 'str'>, <class 'int'>], dtype=object)

Les IRIS de France sont du type 'str' tandis que celui des Collectivités OM sont du type 'int' ou 'float'. On homogénéise le tout en 'str'.

In [11]:
# conversion des IRIS 'int' en 'str'
popglobIR['IRIS'] = popglobIR['IRIS'].astype(str)

In [12]:
# nouvelle verification de types de données IRIS après merge
popglobIR['IRIS'].apply(type).unique()

array([<class 'str'>], dtype=object)

#### All good!
On va procéder à un export afin de le conserver.

In [13]:
# popglobIR.to_csv('IRIS_population/IRIS_population_Fr_CollOM.csv', index=False)

In [14]:
print("\nTotal Population France (hors Mayotte) avec CO-M:", "{:,.2f}".format(popglobIR['P16_POP'].sum()),
      "\nTotal Hommes France (hors Mayotte) avec CO-M :", "{:,.2f}".format(popglobIR['P16_POPH'].sum()),
      "\nTotal Femmes France (hors Mayotte) avec CO-M :", "{:,.2f}".format(popglobIR['P16_POPF'].sum()),
      "\n\nPopulation par IRIS")


Total Population France (hors Mayotte) avec CO-M: 66,413,134.27 
Total Hommes France (hors Mayotte) avec CO-M : 32,137,832.81 
Total Femmes France (hors Mayotte) avec CO-M : 34,275,301.47 

Population par IRIS


# Objectif Population >> DEPT POP ÂGE SEXE
**GEO** : DEPT<br>
**Pop âge**: 0-14 / 15-29 / 30-44 / 45-59 / 60-74 / 75 +<br>
**Pop homme âge**: 0-14 / 15-29 / 30-44 / 45-59 / 60-74 / 75 +<br>
**Pop femme âge**: 0-14 / 15-29 / 30-44 / 45-59 / 60-74 / 75 +<br>

In [15]:
# dataIR = pd.read_csv('IRIS_population/IRIS_population_Fr_CollOM.csv')
dataIR = popglobIR
dataIR

,IRIS,DEP,COM,LIBCOM,TRIRIS,GRD_QUART,LIBIRIS,LAB_IRIS,P16_POP,P16_POP0002,...,C16_F15P_CS5,C16_F15P_CS6,C16_F15P_CS7,C16_F15P_CS8,P16_POP_FR,P16_POP_ETR,P16_POP_IMM,P16_PMEN,P16_PHORMEN,source
0,010010000,01,01001,L'Abergement-Clémenciat,ZZZZZZ,0100100,L'Abergement-Clémenciat (commune non irisée),5,767.000000,29.000000,...,85.000000,30.000000,70.000000,15.000000,759.000000,8.000000,20.000000,767.000000,0.000000,france
1,010020000,01,01002,L'Abergement-de-Varey,ZZZZZZ,0100200,L'Abergement-de-Varey (commune non irisée),5,243.000000,14.000000,...,25.000000,0.000000,20.000000,0.000000,241.000000,2.000000,3.000000,243.000000,0.000000,france
2,010040101,01,01004,Ambérieu-en-Bugey,ZZZZZZ,0100401,Les Perouses-Triangle d'Activité,1,1893.112206,102.492841,...,158.028018,59.452099,216.381754,195.032628,1614.595359,278.516847,308.098676,1557.106066,336.006141,france
3,010040102,01,01004,Ambérieu-en-Bugey,ZZZZZZ,0100401,Longeray-Gare,1,3546.210164,176.709046,...,354.962018,158.716105,351.399678,346.514489,3115.756603,430.453561,515.488156,3546.210164,0.000000,france
4,010040201,01,01004,Ambérieu-en-Bugey,ZZZZZZ,0100402,Centre-St Germain-Vareilles,1,4100.003588,175.616863,...,407.311067,147.066671,529.371210,334.262710,3831.463866,268.539722,341.362897,4012.001980,88.001608,france
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11,978010106,978,97801,Saint-Martin,978231,9780101,Galisbay-Agrement,1,2146.610574,91.364728,...,401.842148,79.088470,95.766908,216.315769,1233.818654,912.791920,883.577973,2146.610574,0.000000,coll_OM
12,978010107,978,97801,Saint-Martin,978231,9780101,C. Gate-St Louis-Rambaud-Colomb.-Pic P.,1,3517.883239,121.104624,...,419.653070,57.729779,174.204029,377.852008,2831.889309,685.993930,832.587752,3517.883239,0.000000,coll_OM
13,978010108,978,97801,Saint-Martin,978241,9780101,La Savane-Gd Case-Anse Marcel-Cul de Sac,1,7399.679871,318.817506,...,918.610483,116.409754,310.436512,797.317626,6251.122404,1148.557466,1444.622295,7399.679871,0.000000,coll_OM
14,978010109,978,97801,Saint-Martin,978241,9780101,Quartier d'Orleans-Gloire-Griselle,1,4462.208206,237.789432,...,536.157672,111.389187,103.116462,805.773743,2751.860160,1710.348046,1646.555314,4462.208206,0.000000,coll_OM


In [16]:
# dataIR.isnull().sum()/len(dataIR)*100

In [17]:
dataIR_base = dataIR[['IRIS', 'DEP', 'COM','LIBCOM', 'GRD_QUART', 'LIBIRIS', 'TRIRIS', 'LAB_IRIS', 'P16_POP', 'P16_POP0002', 'P16_POP0305',
                 'P16_POP0610', 'P16_POP1117', 'P16_POP1824', 'P16_POP2539', 'P16_POP4054', 'P16_POP5564',
                 'P16_POP6579', 'P16_POP80P', 'P16_POP0019', 'P16_POP2064', 'P16_POP65P', 'P16_POPH', 'P16_H0019',
                 'P16_H2064', 'P16_H65P','P16_POPF', 'P16_F0019', 'P16_F2064', 'P16_F65P', 'C16_POP15P', 'C16_H15P', 'C16_H15P_CS1',
                 'C16_H15P_CS2', 'C16_H15P_CS3', 'C16_H15P_CS4', 'C16_H15P_CS5', 'C16_H15P_CS6', 'C16_H15P_CS7',
                 'C16_H15P_CS8', 'C16_F15P', 'C16_F15P_CS1', 'C16_F15P_CS2', 'C16_F15P_CS3', 'C16_F15P_CS4',
                 'C16_F15P_CS5', 'C16_F15P_CS6', 'C16_F15P_CS7', 'C16_F15P_CS8', 'P16_POP_FR', 'P16_POP_ETR', 'P16_POP_IMM',
                 'P16_PMEN', 'P16_PHORMEN']]

### Drop des colonnes dont on n'a pas besoin.

In [18]:
dataIR_new = dataIR.drop(columns=['IRIS','COM', 'LIBCOM', 'GRD_QUART', 'LIBIRIS','TRIRIS', 'LAB_IRIS', 'P16_POP0002', 'P16_POP0305',
                                  'P16_POP0610', 'P16_POP1117', 'P16_POP1824', 'P16_POP2539', 'P16_POP4054', 'P16_POP5564',
                                  'P16_POP6579', 'P16_POP80P', 'P16_POP0019',
                                  'P16_POP2064', 'P16_POP65P', 'P16_H0019', 'C16_H15P_CS1', 'C16_H15P_CS2', 'C16_H15P_CS3', 'C16_H15P_CS4', 'C16_H15P_CS5', 'C16_H15P_CS6',
                                  'C16_H15P_CS7', 'C16_H15P_CS8', 'C16_F15P_CS1', 'C16_F15P_CS2', 'C16_F15P_CS3', 'C16_F15P_CS4',
                                  'C16_F15P_CS5', 'C16_F15P_CS6', 'C16_F15P_CS7', 'C16_F15P_CS8', 
                                  'P16_H2064', 'P16_H65P', 'P16_F0019', 'P16_F2064', 'P16_F65P', 'C16_POP15P', 'C16_H15P', 'C16_F15P', 'C16_POP15P_CS1',
                                  'C16_POP15P_CS2', 'C16_POP15P_CS3', 'C16_POP15P_CS4', 'C16_POP15P_CS5', 'C16_POP15P_CS6', 'C16_POP15P_CS7',
                                  'C16_POP15P_CS8', 'P16_POP_FR', 'P16_POP_ETR', 'P16_POP_IMM',
                                  'P16_PMEN', 'P16_PHORMEN'], axis=1)

In [19]:
dataIR_new.head()

,DEP,P16_POP,P16_POP0014,P16_POP1529,P16_POP3044,P16_POP4559,P16_POP6074,P16_POP75P,P16_POPH,P16_H0014,...,P16_H6074,P16_H75P,P16_POPF,P16_F0014,P16_F1529,P16_F3044,P16_F4559,P16_F6074,P16_F75P,source
0,01,767.000000,161.000000,102.000000,132.000000,189.000000,125.000000,58.000000,392.000000,84.000000,...,61.000000,28.000000,375.000000,77.000000,47.000000,67.000000,90.000000,64.000000,30.000000,france
1,01,243.000000,55.000000,28.000000,70.000000,37.000000,34.000000,19.000000,123.000000,26.000000,...,22.000000,10.000000,120.000000,29.000000,18.000000,32.000000,20.000000,12.000000,9.000000,france
2,01,1893.112206,319.155575,519.011004,346.937127,259.198657,295.470786,153.339057,995.484433,172.326260,...,140.946341,67.457789,897.627773,146.829316,228.196459,150.399826,131.796460,154.524445,85.881268,france
3,01,3546.210164,706.686436,795.052938,737.794854,607.335319,425.641161,273.699455,1742.315872,382.606677,...,168.173042,82.269306,1803.894292,324.079759,395.998460,350.263925,284.653879,257.468120,191.430150,france
4,01,4100.003588,801.240994,848.693971,732.298592,717.244067,591.336732,409.189232,1911.360437,401.716974,...,235.452797,143.979846,2188.643152,399.524021,419.634896,378.848273,369.542642,355.883935,265.209386,france


In [20]:
dataIR_new.apply(lambda x: len(x.unique()))

DEP              103
P16_POP        25645
P16_POP0014    41157
P16_POP1529    40895
P16_POP3044    41172
P16_POP4559    41356
P16_POP6074    41181
P16_POP75P     40662
P16_POPH       41821
P16_H0014      40345
P16_H1529      40019
P16_H3044      40368
P16_H4559      40546
P16_H6074      40392
P16_H75P       39375
P16_POPF       41860
P16_F0014      40252
P16_F1529      39916
P16_F3044      40320
P16_F4559      40573
P16_F6074      40421
P16_F75P       39820
source             2
dtype: int64

In [21]:
# les départements
dataIR_new['DEP'].unique()

array(['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11',
       '12', '13', '14', '15', '16', '17', '18', '19', '21', '22', '23',
       '24', '25', '26', '27', '28', '29', '2A', '2B', '30', '31', '32',
       '33', '34', '35', '36', '37', '38', '39', '40', '41', '42', '43',
       '44', '45', '46', '47', '48', '49', '50', '51', '52', '53', '54',
       '55', '56', '57', '58', '59', '60', '61', '62', '63', '64', '65',
       '66', '67', '68', '69', '70', '71', '72', '73', '74', '75', '76',
       '77', '78', '79', '80', '81', '82', '83', '84', '85', '86', '87',
       '88', '89', '90', '91', '92', '93', '94', '95', '971', '972',
       '973', '974', 975, 977, 978], dtype=object)

In [22]:
# np.count_nonzero(dataIR_new['DEP'].unique(), axis=0)

### Checking missing values
No data are missing!

In [23]:
dataIR_new.isnull().sum()/len(dataIR_new)*100

DEP            0.0
P16_POP        0.0
P16_POP0014    0.0
P16_POP1529    0.0
P16_POP3044    0.0
P16_POP4559    0.0
P16_POP6074    0.0
P16_POP75P     0.0
P16_POPH       0.0
P16_H0014      0.0
P16_H1529      0.0
P16_H3044      0.0
P16_H4559      0.0
P16_H6074      0.0
P16_H75P       0.0
P16_POPF       0.0
P16_F0014      0.0
P16_F1529      0.0
P16_F3044      0.0
P16_F4559      0.0
P16_F6074      0.0
P16_F75P       0.0
source         0.0
dtype: float64

### Agrégation des chiffres par Département

In [24]:
#datadep = dataIR_new.groupby(['DEP']).sum()
datadep = dataIR_new.groupby(['DEP']).sum().reset_index()

In [25]:
#datadep = dataIR_new.groupby(['DEP']).sum()
datadep.shape

(103, 22)

In [26]:
# conversion des DEP 'int' en 'str'
datadep['DEP'] = datadep['DEP'].astype(str)

### Récupération du nom des départements

In [27]:
data = pd.read_json('IRIS_population/areas_fr.json', typ='series')

name = []
for i in datadep['DEP']:
    name.append(data[i])

# inclusion des noms dans une nouvelle colonne
datadep['LIBDEP'] = name

In [28]:
datadep[['DEP','LIBDEP']]

,DEP,LIBDEP
0,975,Saint-Pierre-et-Miquelon
1,977,Saint-Barthelemy
2,978,Saint-Martin
3,01,Ain
4,02,Aisne
...,...,...
98,95,Val-d'oise
99,971,Guadeloupe
100,972,Martinique
101,973,Guyane


In [29]:
datadep.to_csv('IRIS_population/DEP_population_France_CO-M.csv', index=False)

In [30]:
print("\nTotal Population hors Mayotte avec CO-M:", "{:,.2f}".format(datadep['P16_POP'].sum()),
      "\nTotal Hommes hors Mayotte avec CO-M :", "{:,.2f}".format(datadep['P16_POPH'].sum()),
      "\nTotal Femmes hors Mayotte avec CO-M :", "{:,.2f}".format(datadep['P16_POPF'].sum()),
      "\n\nPopulation par REG")


Total Population hors Mayotte avec CO-M: 66,413,134.27 
Total Hommes hors Mayotte avec CO-M : 32,137,832.81 
Total Femmes hors Mayotte avec CO-M : 34,275,301.47 

Population par REG
